IMDB Movie Reviews - Sentiment Analysis

In [56]:
import pandas as pd
import numpy as np

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from tensorflow.keras.models import Model

In [59]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

LOAD IMDB DATASET

In [60]:
import os
os.getcwd()

'C:\\Users\\Asus\\Downloads'

In [61]:
imdb=pd.read_csv('C:/Users/Asus/Downloads/Data/imdb_labelled.txt',sep='\t',header=None,names=['review','sentiment'])
imdb

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0


DATA SPLITTING INTO TRAIN AND TEST

In [62]:
imdb_train,imdb_test=train_test_split(imdb,test_size=0.2,random_state=2)

In [63]:
imdb_train.shape

(598, 2)

In [64]:
imdb_test.shape

(150, 2)

In [65]:
imdb_train

,review,sentiment
456,There still are good actors around!,1
231,"Oh yeah, and the storyline was pathetic too.",0
250,My 8/10 score is mostly for the plot.,1
16,"This review is long overdue, since I consider ...",1
490,I can't see how this movie can be an inspirati...,0
...,...,...
534,") a happy, wonderful, feel good ending!",1
584,"I saw it as a child on TV back in 1973, when i...",1
493,Shot in the Southern California desert using h...,1
527,"Avoid, avoid, avoid!",0


In [66]:
imdb_test

,review,sentiment
62,I'd advise anyone to go and see it.,1
68,"However, this didn't make up for the fact that...",0
603,Avoid at ALL costs!,0
586,This movie is so awesome!,1
192,Though The Wind and the Lion is told largely t...,1
...,...,...
272,Even the squibs look awful.,0
554,Meredith M was better than all right.,1
653,"One of the most boring,pointless movies I have...",0
172,The jerky camera movements were also annoying.,0


In [67]:
#Training Data
x_train=imdb_train['review']
y_train=imdb_train['sentiment']

#Testing Data
x_test=imdb_test['review']
y_test=imdb_test['sentiment']

LENGTH OF THE REVIEWS IN TERMS OF NO.OF WORDS

In [68]:
sent_lens=[]
for sent in imdb_train['review']:
    sent_lens.append(len(word_tokenize(sent)))

In [69]:
max(sent_lens)

1625

FREQUENCY OF REVIEW LENGTH

In [70]:
np.quantile(sent_lens,0.95)

40.0

So, AS WE CAN OBSERVE THAT 95% OF TEXTS ARE OF LENGHTS LESS THAN OR EQUAL TO 40.
WILL CONSIDER FIRST 40 WORDS AND IGNORE THE REST

In [71]:
#Setting maximum number of words 
max_len=40

TOKENIZING THE WORD FROM A REVIEW

In [72]:
tok=Tokenizer(char_level=False,split=' ')

In [73]:
tok.fit_on_texts(x_train)

In [74]:
#A distionary where numeric indices are the keys and the individual words are the values
tok.index_word

{1: 'the',
 2: 'a',
 3: 'and',
 4: 'of',
 5: 'is',
 6: 'this',
 7: 'it',
 8: 'i',
 9: 'to',
 10: 'was',
 11: 'movie',
 12: 'in',
 13: 'film',
 14: 'that',
 15: '1',
 16: '0',
 17: 'but',
 18: 'for',
 19: 'as',
 20: 'with',
 21: 'are',
 22: 'on',
 23: 'not',
 24: 'you',
 25: 'one',
 26: 'very',
 27: 'bad',
 28: 'just',
 29: 'so',
 30: 'good',
 31: 'all',
 32: 'an',
 33: "it's",
 34: 'there',
 35: 'be',
 36: 'by',
 37: 'about',
 38: 'at',
 39: 'if',
 40: 'out',
 41: 'great',
 42: 'his',
 43: 'from',
 44: 'like',
 45: 'have',
 46: 'time',
 47: 'were',
 48: 'well',
 49: 'has',
 50: 'even',
 51: 'really',
 52: 'my',
 53: 'or',
 54: 'who',
 55: 'acting',
 56: 'he',
 57: 'when',
 58: 'most',
 59: 'see',
 60: 'how',
 61: 'more',
 62: 'characters',
 63: 'would',
 64: 'no',
 65: 'only',
 66: 'ever',
 67: 'made',
 68: 'also',
 69: 'best',
 70: '10',
 71: 'plot',
 72: 'some',
 73: 'your',
 74: 'do',
 75: 'its',
 76: 'character',
 77: 'real',
 78: 'because',
 79: 'love',
 80: "didn't",
 81: 'movies

VOCABULARY SIZE

Number of unique/distinct words in the corpus.

The index of the first word in this dictionary is 1.

In [75]:
last_index_in_vocab=len(tok.index_word.keys())

In [76]:
#First Word
tok.index_word[1]

'the'

In [77]:
#length of the Vectorized Word
last_index_in_vocab

2688

In [78]:
#Last Word
tok.index_word[2688]

'passion'

In [79]:
#Assigning the length of Vectorized word in Variable 
vocab_size=len(tok.index_word.keys())
print(vocab_size)

2688


AFTER VECTORIZING, CONVERTING ANY ARBITRARY TEXT TO A SEQUENCE OF INTEGERS

In [80]:
twt=tok.texts_to_sequences(['He is a lazy person.']) #Words not part of the VOCAB will be Dropped eg:'PERSON'
print(twt)

[[56, 5, 2, 613]]


CONVERTING EACH REVIEW TO A SEQUENCE 

In [81]:
#CONVERT THE TRAIN DATA
sequences_train=tok.texts_to_sequences(x_train)

In [82]:
x_train

456                There still are good actors around!  
231       Oh yeah, and the storyline was pathetic too.  
250              My 8/10 score is mostly for the plot.  
16     This review is long overdue, since I consider ...
490    I can't see how this movie can be an inspirati...
                             ...                        
534            ) a happy, wonderful, feel good ending!  
584    I saw it as a child on TV back in 1973, when i...
493    Shot in the Southern California desert using h...
527                               Avoid, avoid, avoid!  
168    I am so pleased to know such a modern day geni...
Name: review, Length: 598, dtype: object

In [83]:
x_train[490]

"I can't see how this movie can be an inspiration to anyone to come out or overcome fear and rejection.  "

In [84]:
y_train

456    1
231    0
250    1
16     1
490    0
      ..
534    1
584    1
493    1
527    0
168    1
Name: sentiment, Length: 598, dtype: int64

In [85]:
#Actual Review Words of the Second Review in the training set
sequences_train[1]

[274, 592, 3, 1, 335, 10, 275, 94]

In [86]:
index=1
review=sequences_train[index]
review_words=[]
for k in review:
    review_words.append((tok.index_word[k]))
review_words

['oh', 'yeah', 'and', 'the', 'storyline', 'was', 'pathetic', 'too']

In [87]:
for word in review_words:
    for key, value in tok.index_word.items():
        if value==word:
            print('{}-{}'.format(key,value))
            break

274-oh
592-yeah
3-and
1-the
335-storyline
10-was
275-pathetic
94-too


PAD SEQUENCES TO MAKE THEM THE SAME SIZE 

In [88]:
sequences_matrix_train=sequence.pad_sequences(sequences_train,maxlen=max_len)
sequences_matrix_train

array([[  0,   0,   0, ...,  30,  90, 426],
       [  0,   0,   0, ...,  10, 275,  94],
       [  0,   0,   0, ...,  18,   1,  71],
       ...,
       [  0,   0,   0, ...,  44,  64,  84],
       [  0,   0,   0, ..., 284, 284, 284],
       [  0,   0,   0, ..., 383,  20, 307]])

In [89]:
sequences_matrix_train.shape

(598, 40)

In [90]:
#Check the size of each review, exactly 40 words (hopefully with zero padding at the beginning)
print(sequences_matrix_train[1])
print('\n Every Input Vector is of Length: ')
print(sequences_matrix_train[1].shape)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 274 592   3   1
 335  10 275  94]

 Every Input Vector is of Length: 
(40,)


CONVERT THE TEST DATA

In [91]:
sequences_test=tok.texts_to_sequences(x_test) # Convert to numeric sequence
sequences_matrix_test=sequence.pad_sequences(sequences_test,maxlen=max_len) #Pad sequences

BUILDING RECURRENT NET WITH LSTM

In [92]:
def RNN():
    inputs=Input(name='inputs',shape=[max_len])
    layer=Embedding(input_dim=vocab_size+1,output_dim=500,input_length=max_len,mask_zero=True)(inputs)
    layer=LSTM(64)(layer)
    layer=Dense(256,name='FC1')(layer)
    layer=Activation('relu')(layer)
    layer=Dropout(0.5)(layer)
    layer=Dense(1,name='out_layer')(layer)
    layer=Activation('sigmoid')(layer)
    model=Model(inputs=inputs,outputs=layer)
    return model

CUSTOM  FUNCTION TO BUILD THE MODEL

In [93]:
model=RNN()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 40)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 40, 500)           1344500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                144640    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [94]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

TRAIN THE MODEL

In [95]:
model.fit(sequences_matrix_train,y_train.values,batch_size=50,epochs=70,validation_data=(sequences_matrix_test,y_test.values))

Epoch 1/70
12/12 [==============================] - 9s 248ms/step - loss: 0.6922 - accuracy: 0.5273 - val_loss: 0.6830 - val_accuracy: 0.5933
Epoch 2/70
12/12 [==============================] - 1s 78ms/step - loss: 0.6709 - accuracy: 0.6951 - val_loss: 0.6228 - val_accuracy: 0.7733
Epoch 3/70
12/12 [==============================] - 1s 80ms/step - loss: 0.4949 - accuracy: 0.9037 - val_loss: 0.5542 - val_accuracy: 0.7667
Epoch 4/70
12/12 [==============================] - 1s 84ms/step - loss: 0.1788 - accuracy: 0.9451 - val_loss: 0.5670 - val_accuracy: 0.7733
Epoch 5/70
12/12 [==============================] - 1s 81ms/step - loss: 0.0552 - accuracy: 0.9916 - val_loss: 0.7261 - val_accuracy: 0.8067
Epoch 6/70
12/12 [==============================] - 1s 80ms/step - loss: 0.0196 - accuracy: 0.9960 - val_loss: 0.8946 - val_accuracy: 0.7933
Epoch 7/70
12/12 [==============================] - 1s 81ms/step - loss: 0.0063 - accuracy: 0.9991 - val_loss: 1.0343 - val_accuracy: 0.8000
Epoch 8/70
1

TEST THE MODEL

In [96]:
predictions=model.predict(sequences_matrix_test)
predictions

array([[1.00000000e+00],
       [9.49198127e-01],
       [7.01496674e-07],
       [9.88873422e-01],
       [1.00000000e+00],
       [6.77202738e-10],
       [1.00000000e+00],
       [9.99976635e-01],
       [2.17797759e-11],
       [2.69064605e-02],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.26543641e-03],
       [9.95256066e-01],
       [1.00000000e+00],
       [4.93494568e-12],
       [1.00000000e+00],
       [8.01328778e-01],
       [1.00000000e+00],
       [2.44035125e-10],
       [2.97551495e-09],
       [9.99999523e-01],
       [1.00000000e+00],
       [9.99999702e-01],
       [5.60836377e-10],
       [3.12155771e-06],
       [1.18617879e-06],
       [8.22513103e-01],
       [9.99963164e-01],
       [2.88290812e-06],
       [9.99998808e-01],
       [1.00000000e+00],
       [3.78429890e-04],
       [1.00000000e+00],
       [2.03464047e-12],
       [5.19553556e-09],
       [9.86231399e-08],
       [1.04717355e-08],
       [1.00000000e+00],
       [4.74121400e-08],


EVALUATION WITH ROC-AUC SCORE

In [97]:
from sklearn.metrics import roc_auc_score

In [98]:
roc_auc_score(y_test,predictions)

0.8084630524244051

MAKE A PREDICTION

In [103]:
# Choose a review from the test set (any index number)
index=1
review=sequences_test[index]
review_words=[]
for k in review:
    review_words.append((tok.index_word[k]))
review_words

['however',
 'this',
 "didn't",
 'make',
 'up',
 'for',
 'the',
 'fact',
 'that',
 'overall',
 'this',
 'was',
 'a',
 'tremendously',
 'boring',
 'movie']

In [104]:
model.predict(sequences_matrix_test[index].reshape(1,40))
pred = model.predict(sequences_matrix_test[index].reshape(1,40))[0][0]
print(pred)

0.94919825
